In [1]:
from sagas.nlu.utils import fix_sents
import requests
from sagas.conf.conf import cf

def fix_data(data):
    if 'engine' not in data:
        data['engine'] = cf.engine(data['lang'])
    data['sents']=fix_sents(data['sents'], data['lang'])
    return data
def parse(data):
    if 'engine' not in data:
        data['engine']=cf.engine(data['lang'])
    engine=data['engine']
    response = requests.post(f'{cf.servant(engine)}/verb_domains', json=data)
    if response.status_code == 200:
        return response.json()
    return None

In [2]:
# $ ses 'Ellos ya leyeron ese libro en la escuela.'
# data={'lang': 'es', "sents": 'Ellos ya leyeron ese libro en la escuela.'}
# $ se 'you are dead'
# data={'lang': 'en', "sents": 'you are dead'}
# $ sid 'Apa yang lebih murah?'  (What is cheaper?)
# data={'lang': 'id', "sents": 'Apa yang lebih murah?'}
# $ sid 'Berapa umur kamu?' (en="How old are you?")
# data={'lang': 'id', "sents": 'Berapa umur kamu?'}
# $ sj '太陽は月に比べて大きいです。'
# $ sj '予約を火曜日から木曜日に変えてもらった。'
data={'lang': 'ja', "sents": '予約を火曜日から木曜日に変えてもらった。'}
rs=parse(data)
rs

[{'type': 'predicate',
  'lemma': '変える',
  'index': 3,
  'phonetic': 'かえる',
  'word': '変えてもらった。',
  'rel': 'D',
  'governor': 0,
  'pos': 'verb',
  'domains': [['ヲ', 0, '予約を', '予約', ['予約'], ['c_noun', 'x_n']],
   ['時間', 1, '火曜日から', '火曜日', ['火曜日'], ['c_noun', 'x_n']],
   ['時間', 2, '木曜日に', '木曜日', ['木曜日'], ['c_noun', 'x_n']]],
  'stems': [],
  'segments': [{'index': 6,
    'upos': 'VERB',
    'xpos': ['動詞', '*'],
    'text': '変えて',
    'lemmas': ['変える', 'かえる']},
   {'index': 7,
    'upos': 'X',
    'xpos': ['接尾辞', '動詞性接尾辞'],
    'text': 'もらった',
    'lemmas': ['もらう', 'もらう']},
   {'index': 8, 'upos': 'PUNCT', 'xpos': ['特殊', '句点'], 'text': '。'}]}]

In [3]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
def to_obj(m):
    d = AttrDict()
    d.update(m)
    return d

In [6]:
from sagas.nlu.rules_meta import build_meta
from sagas.nlu.rules_lang_spec import langspecs
from sagas.nlu.sinkers import Sinkers
from sagas.tool.misc import translit_chunk, display_synsets
import sagas.tracker_fn as tc
from sagas.nlu.utils import fix_sents, join_text
import sagas
from pprint import pprint
from collections import Counter

enable_verbose=True
def proc_word(type_name, word, head, index, lang):
    from sagas.nlu.google_translator import translate
    res, _ = translate(word, source=lang, target=target_lang(lang),
                       trans_verbose=False)
    
    # result=f"[{type_name}]({word}{translit_chunk(word, lang)}) {res}{target}"
    result={'type':type_name, 'text':word, 
            'translit':translit_chunk(word, lang),
            'translate': res,
            'lang': lang,
            'index': index,
           }
    if head!='':
        res_t, _ = translate(head, source=lang, target=target_lang(lang),
                           trans_verbose=False, options={'disable_correct'})
        # target=f" ⊙︿⊙ {res_t}({head})"
        result['head']=head
        result['head_trans']=res_t
    # tc.emp('magenta', result)
    return result
target_lang=lambda s: cf.optional('assist_lang', 'zh') if s=='en' else 'en'

# def proc_children_column(partcol, textcol, idxcol, lang):
def proc_children_column(df, lang):
    from sagas.nlu.google_translator import translate
    result=[]
    # for id, (name, r) in enumerate(zip(partcol, textcol)):
    rels=[]
    for id, row in df.iterrows(): 
        # df['rel'], df['children'], df['index']
        name, r, idx=row['rel'], row['children'], row['index']
        if name in rels:
            continue
        else:
            rels.append(name)
        if name not in ('punct', 'head_root'):        
            sent=join_text(r, lang)
            res, _ = translate(sent, source=lang, target=target_lang(lang),
                               trans_verbose=False, options={'disable_correct'})
            # chunk=f"{indent}[{name}]({sent}{translit_chunk(sent, lang)}) {res}"
            chunk={'name':name, 'text':sent, 
                   'translit':translit_chunk(sent, lang),
                   'translate':res,
                   'index':idx,
                  }
            result.append(chunk)
            # tc.emp('cyan', chunk)
    return result

def induce_spec(el, pats, type_name):
    # print(f"{el.indicator} {el.word}: {el.spec}")    
    if el.indicator=='[verb]':
        pats.append((3, f"behaveof('{el.spec}', 'v')"))
    elif el.indicator=='[aux]':
        # pats.append((3, f"behaveof('{el.spec}', '*')"))
        pass
    elif el.indicator=='[root]':
        pats.append((3, f"behaveof('{el.spec}', 'n')"))
    elif el.indicator=='[predicate]':
        pats.append((3, f"specsof('*', '{el.spec}')"))
    else:
        if type_name in ('aux_domains', 'subj_domains') and el.indicator=='head':
            pats.append((3, f"behaveof('{el.spec}', '*')"))
        else:
            pats.append((1, f"{el.indicator}=kindof('{el.spec}', '*')"))

def induce_part(chunk, pats, type_name):
    if enable_verbose:
        tc.emp('cyan', chunk)
    gen_map={'nsubj': lambda: (2, "nsubj=agency"), 
             'advmod': lambda: (4, "extract_for('plain', 'advmod')"),
             'cop': lambda: (2, "cop='c_aux'"),
             'head_amod': lambda: (2, "head_amod=interr('what')"),
             '時間': lambda: (4, "extract_for('plain+date_search+date_parse', '時間')"),
            }
    if chunk.name in gen_map:
        pats.append(gen_map[chunk.name]())

def induce_pattern(pat, ds):
    if enable_verbose:
        tc.emp('magenta', pat)    
    def gen_verb(ind='verb', prefix='behave'):
        spec=[d for d in ds if d['indicator']==f'[{ind}]']
        if spec:
            ref=spec[0]['spec']
        else:
            ref=pat.translate.replace(' ', '_')
        return f"pat(5, name='{prefix}_{ref}').verb"
    def gen_root():
        spec=[d for d in ds if d['indicator']=='[root]']
        if spec:
            ref=spec[0]['spec']
        else:
            ref=pat.translate.replace(' ', '_')
        return f"pat(5, name='ana_{ref}').root"
    def gen_cop():
        spec=[d for d in ds if d['indicator']=='head']
        if spec:
            ref=spec[0]['spec']
        else:
            ref=pat.head_trans.replace(' ', '_') if pat.lang!='en' else pat.head.replace(' ', '_')
        return f"pat(5, name='desc_{ref}').cop"    
    
    domap={'verb_domains': gen_verb,
           'aux_domains': gen_cop,
           'subj_domains': gen_cop,
           'root_domains': gen_root,
           'predicate': lambda: gen_verb('predicate', 'predict'),
          }
    return domap[pat.type]()

######
sinkers = Sinkers()
for serial, r in enumerate(rs):
    type_name = r['type']
    theme=type_name.split('_')[0]
    print(type_name)
    meta = build_meta(r, data)
    # print(f"meta keys {meta.keys()}")
    
    mod_rs=langspecs.check_langspec(data['lang'], meta, r['domains'], type_name)
    sinkers.add_module_results(mod_rs)
    
    # infers
    pats=[]
    def do_infers(ds):
        df = sagas.to_df(r['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])

        pat=proc_word(type_name, r['word'], 
                      r['head'] if 'head' in r else '', 
                      r['index'],
                      data['lang'])
        pat_r=induce_pattern(to_obj(pat), ds)
        parts=proc_children_column(df, data['lang'])    
        for part in parts:
            induce_part(to_obj(part), pats, type_name)
        # display_synsets(f"[{theme}]", meta, r, data['lang'])    
        return pat_r
    
    # induce with wordnet
    ds=display_synsets(f"[{theme}]", meta, r, data['lang'], collect=True)    
    pat_r=do_infers(ds)
    for el in ds:        
        induce_spec(to_obj(el), pats, type_name)
    
    pats=sorted(pats, key=lambda pat: -pat[0])
    paras=', '.join(p[1] for p in pats)
    print(f"{pat_r}({paras}),")
    
    # debug
    print('*', '-'*45)
    pprint(ds)

sinkers.process_with_sinkers()

predicate
✁ lang.spec for ja.predicate -------------------------
✖ (desc_fav) verb with pos is ins_interrogative(fav): False
✖ (desc_fav_not) verb with pos is ins_interrogative(fav): False, pos is ins_checker{'has_lemma': 'ない'}: False
✖ (desc_fav_neg) verb with pos is ins_interrogative(fav): False, pos is ins_checker{'negative': '_'}: False
✖ (desc_attitude_type) verb with pos is specs_of(('tight',),*): False
✔ (change_statement) verb with pos is specs_of(('change',),v): True, ヲ is kind_of(statement,n): True, pos is ins_extract_comps(['plain', 'date_search', 'date_parse']): True
.. results 8
{'extract_comps/date_parse/時間', 'extract_comps/date_search/時間', 'specs_of/default/predicate', 'kind_of/default/ヲ', 'extract_comps/plain/時間'}
{'specs_of/default/...': {'category': 'change', 'pos': 'v', 'word': '変えてもらった。/変える'}, 'kind_of/default/...': {'category': 'statement', 'pos': 'n', 'word': '予約を/予約'}, 'extract_comps/plain/...': '木曜日', 'extract_comps/date_search/...': ["('木曜日', datetime.datetime(

In [5]:
pickup=''
'true' if pickup else 'false'

'false'